In [1]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers
from keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from time import time

In [2]:
train_data = pd.read_csv('E:/Python/data/twitter/train.csv')
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
#gpu memory growth fix
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [5]:
X_train = []
sentences = list(train_data['text'])
for sen in sentences:
    X_train.append(preprocess_text(sen))

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=22128)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(train_data['text'])
X_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
vocab_size = len(tokenizer.word_index) + 1
maxlen=100
# X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

In [7]:
vocab_size

22128

In [18]:
X_train.shape

(7613, 22128)

In [9]:
# X_train = X_train.reshape((7613, 100, 1))

In [10]:
# # determine the number of input features
# # n_features = 100
# # define model
# model = Sequential()
# model.add(layers.LSTM(128, input_shape=(X_train.shape[1], 1)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(2, activation='softmax'))
# # compile the model
# model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
# #callbacks
# # simple early stopping
# # es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=TOL, verbose=VERBOSE, patience=N_ITER_NO_CHANGE)
# # mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=VERBOSE, save_best_only=True)

# tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
# # fit the model
# model.fit(X_train, train_data['target'], epochs=100, batch_size=32, callbacks=[tensorboard,])
# # make a prediction
# # yhat = model.predict(test_X_enc)

In [17]:
# define model
model = Sequential()
model.add(layers.Dense(512, kernel_initializer='he_normal', input_shape=(vocab_size,)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
#callbacks
# simple early stopping
# es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=TOL, verbose=VERBOSE, patience=N_ITER_NO_CHANGE)
# mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=VERBOSE, save_best_only=True)

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
# fit the model
model.fit(X_train, train_data['target'], epochs=100, batch_size=16, callbacks=[tensorboard,], validation_split=0.2)
# make a prediction
# yhat = model.predict(test_X_enc)

ResourceExhaustedError: OOM when allocating tensor with shape[22128,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:TruncatedNormal]